In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
import logging
import numpy as np
import py21cmfast as p21c
import math
from py21cmfast import (
    AstroParams,
    CosmoParams,
    FlagOptions,
    UserParams,
    determine_halo_list,
    initial_conditions,
    perturb_field,
    plotting
    )

/opt/anaconda3/envs/21cmfast/lib/python3.9/site-packages/py21cmfast/_cfg.py:49: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/opt/anaconda3/envs/21cmfast/lib/python3.9/site-packages/py21cmfast/_cfg.py:33: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


In [13]:
import h5py
import numpy as np

def load_data_from_h5(filename):

    with h5py.File(filename, "r") as hf:

        print("Datasets: ", list(hf.keys()))

        X = hf["X"][:]
        Y = hf["Ytrue"][:]
        Ypred = hf["Ypred"][:]
        redshifts = hf["redshifts"][:]

    return X, Y, Ypred, redshifts

results_file = '/Users/jacobkennedy/Desktop/PHYS_459/092021/sept_20_test_boxes/z8-9_HII-DIM-128_BOX-LEN-256_coeval_boxes_results.h5'

'''

X: The wedge-filtered coeval boxes
Y: The original coeval boxes, generated by 21cm Fast
Y_pred: The lightcones that the machine learning model predicts.
redshifts: The redshifts that the lightcones were generated at

'''

X, Y, Ypred, redshifts = load_data_from_h5(results_file)


print("X shape: ", X.shape)

Datasets:  ['X', 'Ypred', 'Ytrue', 'redshifts']
X shape:  (2, 128, 128, 128)


In [14]:
initial_conditions = p21c.initial_conditions(
    user_params = {"HII_DIM": 128, "BOX_LEN": 128}, 
    cosmo_params = p21c.CosmoParams(SIGMA_8 = 0.8), 
    random_seed = 42242)

/opt/anaconda3/envs/21cmfast/lib/python3.9/site-packages/py21cmfast/inputs.py:485: UserWarning: The USE_INTERPOLATION_TABLES setting has changed in v3.1.2 to be default True. You can likely ignore this warning, but if you relied onhaving USE_INTERPOLATION_TABLES=False by *default*, please set it explicitly. To silence this warning, set it explicitly to True. Thiswarning will be removed in v4.
  warnings.warn(


In [15]:
print(redshifts)

[8.   8.25 8.5 ]


In [17]:
redshift = 8.0

pt_box = perturb_field(
    redshift=redshift,
    init_boxes=initial_conditions
    )

halos = determine_halo_list(
    redshift=redshift,
    init_boxes=initial_conditions,
    regenerate=True,
    write=False,
    OPTIMIZE=False,
    )